### Model Installation & Module Importion

In [1]:
!pip install -q git+https://github.com/openai/whisper.git
!pip install -U transformers accelerate
!pip install gTTS
!pip install huggingface_hub
!pip install ipython

import warnings
from gtts import gTTS
from IPython.display import Audio, display
from huggingface_hub import login
from transformers import pipeline
import whisper
warnings.filterwarnings("ignore")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 20.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.6 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcugraph-cu12 24.12.0 requires pylibraft-cu12=

2025-05-09 13:20:30.750454: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746796831.036925      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746796831.121092      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### Speech-To-Text

In [2]:
def load_whisper_model(model_size="small"):
    try:
        return whisper.load_model(model_size)
    except Exception as e:
        raise RuntimeError(f"Failed to load Whisper model: {str(e)}")

whisper_model = load_whisper_model()

def transcribe_audio(file_path: str) -> str:
    try:
        result = whisper_model.transcribe(file_path)
        return result['text'].strip()
    except Exception as e:
        print(f"Transcription Error: {str(e)}")
        return ""

100%|███████████████████████████████████████| 461M/461M [00:05<00:00, 94.7MiB/s]


### Emotion Detection

In [3]:
def initialize_emotion_classifier():
    try:
        login(" ") # Add HF Secret here
        return pipeline("text-classification", model="arpanghoshal/EmoRoBERTa")
    except Exception as e:
        raise RuntimeError(f"Failed to initialize emotion classifier: {str(e)}")

emotion_classifier = initialize_emotion_classifier()

def detect_emotion(text: str) -> str:
    if not text.strip():
        return "neutral"
    try:
        result = emotion_classifier(text)[0]
        print(f"Detected Emotion: {result['label']} (confidence: {result['score']:.2f})")
        return result['label'].lower()
    except Exception as e:
        print(f"Emotion detection failed: {str(e)}")
        return "neutral"

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/501M [00:00<?, ?B/s]

2025-05-09 13:21:07.275090: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at arpanghoshal/EmoRoBERTa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use 0


### Response Generation

In [48]:
def initialize_llm_pipeline():
    try:
        generator = pipeline(
            "text-generation",
            model="tiiuae/falcon-rw-1b",
            max_new_tokens=100,
            temperature=0.2,
            do_sample=True,
            device_map="auto"
        )
        generator.tokenizer.pad_token_id = generator.model.config.eos_token_id
        return generator
    except Exception as e:
        raise RuntimeError(f"Failed to initialize LLM pipeline: {str(e)}")
llm = initialize_llm_pipeline()

def generate_empathic_response(transcribed_text: str, emotion: str) -> str:
    """Generates natural, human-like emotional responses"""
    if not transcribed_text.strip():
        return "I'm here to listen."
    prompt = f"""**Context**: A person shared:"{transcribed_text}", **Their Emotion is**:"{emotion}". Craft a friendly response of upto 100 words.
**Response**:"""
    try:
        response = llm(prompt, max_length=100, num_return_sequences=1)[0]['generated_text']
        clean_response = response.split("**Response**:")[-1].strip()
        clean_response = clean_response.split("\n\n")[0].strip()
        clean_response = clean_response.replace('"', '') 
        if not clean_response.endswith(('.','!','?')):
            clean_response += '.'
        return clean_response
    except Exception as e:
        print(f"Response error: {e}")
        return "This pain deserves to be witnessed. Thank you for sharing it."

Device set to use cpu


### Text-To-Speech

In [49]:
def speak_response(text: str, lang='en', filename='response.mp3'):
    if not text.strip():
        return None
    try:
        tts = gTTS(text=text, lang=lang, slow=False)
        tts.save(filename)
        print(f"🗣️ Voice response saved as '{filename}'")
        
        # Create and display the audio widget
        audio_widget = Audio(filename, autoplay=True)
        display(audio_widget)
        return audio_widget
    except Exception as e:
        print(f"TTS Error: {str(e)}")
        return None

### Main Workflow with an example

In [50]:
def process_audio_to_speech(file_path: str):
    print("\nProcessing audio file...")
    transcribed_text = transcribe_audio(file_path)
    if not transcribed_text:
        print("No text was transcribed from the audio.")
        return
    print(f"\nTranscribed Text: {transcribed_text}")
    emotion = detect_emotion(transcribed_text)
    print("\nGenerating response...")
    response = generate_empathic_response(transcribed_text, emotion)
    print(f"\nModulo Says: {response}")
    print("\nConverting to speech...")
    speak_response(response)

# Example
if __name__ == "__main__":
    audio_file = "/kaggle/input/ratantata/ytmp3free.cc_emotional-speech-ever-ratan-tata-youtubemp3free.org.mp3"
    process_audio_to_speech(audio_file)


Processing audio file...

Transcribed Text: Both my mother and my father died of cancer. Millions of people are struck by cancer, been mistreated, wrongly treated, treated too late. The country has had poor facilities to deal with this. The National Cancer Institute is an unbelievable jewel that's been created in Nagpur to fight this disease. The Tata Trust will support this and the whole trust that the Government of India is giving to cancer because it is a scourge that we can control over the years.


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=100) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Detected Emotion: sadness (confidence: 0.72)

Generating response...

Modulo Says: I am a cancer survivor. I have been treated at the National Cancer Institute in Nagpur. I have been treated by the best doctors. I have been treated with the best medicines. I am a cancer survivor. I have been treated at the National Cancer Institute in Nagpur. I have been treated by the best doctors. I have been treated with the best medicines. I am a cancer survivor. I have been treated at the National Cancer Institute in Nagpur. I have been treated with.

Converting to speech...
🗣️ Voice response saved as 'response.mp3'
